In [ ]:
from datetime import datetime 
import pandas as pd
import numpy as np
import re
import emoji

# The format string should match the format of the date and time string
# In this case, the date and time string is in the format 'MM/DD/YY, HH:MM:SS AM/PM'
date_time_format = '%m/%d/%y, %I:%M:%S %p'
date_time_list = []
name_list = []
text_list = []
total_file = []

file = r"C:\Users\markkw\Downloads\WhatsApp Chat\_chat.txt"
with open(file, encoding="utf8") as f:
    for line in f.readlines():
        total_file.append(line)
        if line[0] == "[":
            date_time_list.append(datetime.strptime(line.split(']', 1)[0].replace('[', ''), date_time_format))
            name_list.append(line.split(']', 1)[1].split(":",1)[0])
            if ":" in line.split(']', 1)[1]:
                text_list.append(line.split(']', 1)[1].split(":",1)[1].replace("\n", ""))
            else: 
                text_list.append("-")

df = pd.DataFrame(index=[date_time_list])
df['Person'] = name_list
df['Message'] = text_list
df = df[df['Message'] != "-"]
pattern = re.compile(r"|".join(map(re.escape, emoji.EMOJI_DATA)))

df["Emojis"] = df["Message"].apply(lambda x: "".join(pattern.findall(x)))
df["Sentence"] = df["Message"].apply(lambda x: pattern.sub("", x))



In [ ]:
import translators.server as tss
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

def sentiment_analyzer(text):
    from_language, to_language = 'nl', 'en'
    try:
        translated_text = tss.google(text['Sentence'], from_language, to_language) + text['Emojis']
    except:
        translated_text = text['Sentence']
    score = analyser.polarity_scores(translated_text)
    
    return score['compound']

In [ ]:
df_test = df.iloc[:200,:]
df_test['Sentiment'] = df_test.apply(lambda row : sentiment_analyzer(row), axis=1)
df['Emojis'].value_counts()